In [1]:
# ---------- visualizzo piu output a schermo insieme ------------------------------
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
#-------------------------- scelgo se usare CPU o GPU -----------------------------
import tensorflow as tf
import numpy as np
import os

SEED = 1234
tf.random.set_seed(SEED)
cwd = os.getcwd()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0' # CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
elif cpus:
    try:
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        print(e)
#---------------------------------------------------------------------------

1 Physical GPUs, 1 Logical GPUs


In [2]:
import json, shutil

with open('MaskDataset\\train_gt.json', 'r') as labels:
    data=labels.read()
    
obj = json.loads(data)

X = []
Y = []

for key, value in obj.items():
    X.append(key)
    Y.append(value)
    
X=np.array(X)
Y=np.array(Y)

In [9]:

def CreateModel():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(10, input_shape=(4,) , activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation = 'relu'))
    model.add(tf.keras.layers.Dense(3, activation = 'softmax'))
    
    model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )
    
    return model


In [10]:
img_h=299
img_w=299

In [11]:
from sklearn.model_selection import KFold

n_split=3

for train_index,test_index in KFold(n_split).split(X):
    x_train,x_test=X[train_index],X[test_index]
    y_train,y_test=Y[train_index],Y[test_index]
    
    model=CreateModel()
    model.fit(x_train, y_train,epochs=20)
    print('Model evaluation ',model.evaluate(x_test,y_test))
    
   
    
    
    
    

ValueError: Error when checking input: expected dense_4_input to have shape (4,) but got array with shape (1,)

In [ ]:
#------------- salvo i pesi del modello -------------------------------------------------------
save_weights = True

if save_weights:
    model_name = 'server'
    base_name = 'base_weights'
    model_dir = os.path.join(cwd, 'BIG_SERVER\\BIG_SERVER')
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)    
    model.save_weights(os.path.join(model_dir, model_name))
    
    base_weights = []    
    for i in range(3):
        base_weights.append(base.layers[-4+i].get_weights())
    
    np.save(os.path.join(model_dir, base_name),base_weights)



In [ ]:
# directory dove salvoi i risultati (di entrambi i modelli)
results_dir = os.path.join(cwd, 'results')
if not os.path.exists(results_dir):
        os.makedirs(results_dir)

results_dir = os.path.join(results_dir, 'BIG_SERVER')
if not os.path.exists(results_dir):
        os.makedirs(results_dir)

#------------ funzione per formattare il file csv ---------------------------------------------
def create_csv(results, results_dir=results_dir):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
#--------- scorro le immagini di test e faccio predizioni --------------------------------------
from PIL import Image
from keras.preprocessing import image

imm_source = os.path.join(cwd, 'MaskDataset\\test')
image_filenames = next(os.walk(imm_source))[2]
# scegli tra questi label per le predizioni
labels=['0', '1', '2']

results = {}
for image_name in image_filenames:   
    img = image.load_img(os.path.join(imm_source, image_name), target_size=(img_h, img_w))    
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.    
    pred = model.predict(img_tensor)
    results[image_name] = labels[np.argmax(pred)]
    
create_csv(results)
